## BBC News Data Preprocessing

In [570]:
import pandas as pd
import os
import glob
import keras

from nltk.tokenize import RegexpTokenizer

In [571]:
#Assumes data is in current directory

articles_path = "BBC News Summary/News Articles"
summaries_path = "BBC News Summary/Summaries"

business_articles = articles_path + "/business"
business_summaries = summaries_path + "/business"

entertainment_articles = articles_path + "/entertainment"
entertainment_summaries = summaries_path + "/entertainment"

politics_articles = articles_path + "/politics"
politics_summaries = summaries_path + "/politics"

sport_articles = articles_path + "/sport"
sport_summaries = summaries_path + "/sport"

tech_articles = articles_path + "/tech"
tech_summaries = summaries_path + "/tech"

paths = [[business_articles, business_summaries], 
         [entertainment_articles, entertainment_summaries],
         [politics_articles, politics_summaries],
         [sport_articles, sport_summaries],
         [tech_articles, tech_summaries]]

data = []

for articles, summaries in paths:
    for article, summary in zip(os.listdir(articles), os.listdir(summaries)):
        with(open(articles + "/" + article, "r")) as f1:
            with(open(summaries + "/" + summary, "r")) as f2:
                data.append([f1.read(), f2.read()])
            

In [572]:
df = pd.DataFrame(data, columns=["article", "summary"])

In [573]:
df.head()

,article,summary
0,Ad sales boost Time Warner profit\n\nQuarterly...,TimeWarner said fourth quarter sales rose 2% t...
1,Dollar gains on Greenspan speech\n\nThe dollar...,The dollar has hit its highest level against t...
2,Yukos unit buyer faces loan claim\n\nThe owner...,Yukos' owner Menatep Group says it will ask Ro...
3,High fuel prices hit BA's profits\n\nBritish A...,"Rod Eddington, BA's chief executive, said the ..."
4,Pernod takeover talk lifts Domecq\n\nShares in...,Pernod has reduced the debt it took on to fund...


In [574]:
def clean(sentence):
    sentence = str(sentence).lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    return tokens

In [575]:
clean_articles = []
for entry in df['article']:
    clean_articles.append(clean(entry))

clean_summaries = []
for entry in df['summary']:
    clean_summaries.append(clean(entry))

In [576]:
vocab = list(set([word for article in clean_articles for word in article]
                 + [word for summary in clean_summaries for word in summary]))

In [577]:
empty = 0 # RNN mask of no data
eos = 1  # end of sentence
start_idx = eos+1 # first real word

word2idx = dict((word, idx+start_idx) for idx,word in enumerate(vocab))
word2idx['<empty>'] = empty
word2idx['<eos>'] = eos
    
idx2word = dict((idx,word) for word,idx in word2idx.items())

In [578]:
embedding_dim = 100
glove_n_symbols = 400000
glove_name = "glove.6B.100d.txt"

import numpy as np
glove_index_dict = {}
glove_embedding_weights = np.empty((glove_n_symbols, embedding_dim))
globale_scale=.1
with open(glove_name, 'r', encoding="utf8") as fp:
    i = 0
    for l in fp:
        l = l.strip().split()
        w = l[0]
        glove_index_dict[w] = i
        glove_embedding_weights[i,:] = [float(v) for v in l[1:]]
        i += 1
glove_embedding_weights *= globale_scale

In [579]:
for w,i in glove_index_dict.items():
    w = w.lower()
    if w not in glove_index_dict:
        glove_index_dict[w] = i

In [580]:
vocab_size = len(vocab)

# generate random embedding with same scale as glove
np.random.seed(0)
shape = (vocab_size, embedding_dim)
scale = glove_embedding_weights.std()*np.sqrt(12)/2 # uniform and not normal
embedding = np.random.uniform(low=-scale, high=scale, size=shape)

# copy from glove weights of words that appear in our short vocabulary (idx2word)
c = 0
for i in range(vocab_size):
    w = idx2word[i]
    g = glove_index_dict.get(w, glove_index_dict.get(w.lower()))
    if g is None and w.startswith('#'): # glove has no hastags (I think...)
        w = w[1:]
        g = glove_index_dict.get(w, glove_index_dict.get(w.lower()))
    if g is not None:
        embedding[i,:] = glove_embedding_weights[g,:]
        c+=1

In [581]:
glove_thr = 0.5

In [582]:
word2glove = {}
for w in word2idx:
    if w in glove_index_dict:
        g = w
    elif w.lower() in glove_index_dict:
        g = w.lower()
    elif w.startswith('#') and w[1:] in glove_index_dict:
        g = w[1:]
    elif w.startswith('#') and w[1:].lower() in glove_index_dict:
        g = w[1:].lower()
    else:
        continue
    word2glove[w] = g

In [583]:
normed_embedding = embedding/np.array([np.sqrt(np.dot(gweight,gweight)) for gweight in embedding])[:,None]

nb_unknown_words = 100

glove_match = []
for w,idx in word2idx.items():
    if idx >= vocab_size-nb_unknown_words and w.isalpha() and w in word2glove:
        gidx = glove_index_dict[word2glove[w]]
        gweight = glove_embedding_weights[gidx,:].copy()
        # find row in embedding that has the highest cos score with gweight
        gweight /= np.sqrt(np.dot(gweight,gweight))
        score = np.dot(normed_embedding[:vocab_size-nb_unknown_words], gweight)
        while True:
            embedding_idx = score.argmax()
            s = score[embedding_idx]
            if s < glove_thr:
                break
            if idx2word[embedding_idx] in word2glove :
                glove_match.append((w, embedding_idx, s)) 
                break
            score[embedding_idx] = -1
glove_match.sort(key = lambda x: -x[2])

In [584]:
glove_idx2idx = dict((word2idx[w],embedding_idx) for  w, embedding_idx, _ in glove_match)

In [585]:
maxlend=0 # 0 - if we dont want to use description at all
maxlenh=25
maxlen = maxlend + maxlenh
rnn_size = 512 # must be same as 160330-word-gen
rnn_layers = 3  # match FN1
batch_norm=False

activation_rnn_size = 40 if maxlend else 0

p_W, p_U, p_dense, p_emb, weight_decay = 0, 0, 0, 0, 0
optimizer = 'adam'
LR = 1e-4
batch_size=64
nflips=10

vocab_size, embedding_size = embedding.shape

X = [[word2idx[token] for token in a] for a in clean_articles]
Y = [[word2idx[token] for token in s] for s in clean_summaries]

In [606]:
nb_val_samples = 20
nb_train_samples = 20

nb_unknown_words = 10

for i in range(nb_unknown_words):
    idx2word[vocab_size-1-i] = '<%d>'%i
    
oov0 = vocab_size-nb_unknown_words

for i in range(oov0, len(idx2word)):
    idx2word[i] = idx2word[i]+'^'

In [607]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=nb_val_samples)
len(X_train), len(Y_train), len(X_test), len(Y_test)


NameError: name 'X' is not defined

In [608]:
del X
del Y

NameError: name 'X' is not defined

In [609]:
empty = 0
eos = 1
idx2word[empty] = '_'
idx2word[eos] = '~'

In [610]:
import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils
import random, sys

In [611]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, RepeatVector#, Merge
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.regularizers import l2

In [612]:
random.seed(0)
np.random.seed(0)

In [613]:
regularizer = l2(weight_decay) if weight_decay else None

In [614]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_size,
                    input_length=maxlen,
                    W_regularizer=regularizer, dropout=p_emb, weights=[embedding], mask_zero=True,
                    name='embedding_1'))
for i in range(rnn_layers):
    lstm = LSTM(rnn_size, return_sequences=True, # batch_norm=batch_norm,
                W_regularizer=regularizer, U_regularizer=regularizer,
                b_regularizer=regularizer, dropout_W=p_W, dropout_U=p_U,
                name='lstm_%d'%(i+1)
                  )
    model.add(lstm)
    model.add(Dropout(p_dense,name='dropout_%d'%(i+1)))

C:\Users\rasik\Anaconda3\envs\mlenv\lib\site-packages\ipykernel\__main__.py:5: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
C:\Users\rasik\Anaconda3\envs\mlenv\lib\site-packages\ipykernel\__main__.py:5: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(29467, 100, input_length=25, weights=[array([[ ..., mask_zero=True, name="embedding_1", embeddings_regularizer=None)`
C:\Users\rasik\Anaconda3\envs\mlenv\lib\site-packages\ipykernel\__main__.py:10: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequences=True, name="lstm_1", kernel_regularizer=None, bias_regularizer=None, recurrent_regularizer=None, dropout=0, recurrent_dropout=0)`
C:\Users\rasik\Anaconda3\envs\mlenv\lib\site-packages\ipykernel\__main__.py:10: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequences=True

In [615]:
from keras.layers.core import Lambda
import keras.backend as K

def simple_context(X, mask, n=activation_rnn_size, maxlend=maxlend, maxlenh=maxlenh):
    desc, head = X[:,:maxlend,:], X[:,maxlend:,:]
    head_activations, head_words = head[:,:,:n], head[:,:,n:]
    desc_activations, desc_words = desc[:,:,:n], desc[:,:,n:]
    
    # RTFM http://deeplearning.net/software/theano/library/tensor/basic.html#theano.tensor.batched_tensordot
    # activation for every head word and every desc word
    activation_energies = K.batch_dot(head_activations, desc_activations, axes=(2,2))
    # make sure we dont use description words that are masked out
    activation_energies = activation_energies + -1e20*K.expand_dims(1.-K.cast(mask[:, :maxlend],'float32'),1)
    
    # for every head word compute weights for every desc word
    activation_energies = K.reshape(activation_energies,(-1,maxlend))
    activation_weights = K.softmax(activation_energies)
    activation_weights = K.reshape(activation_weights,(-1,maxlenh,maxlend))

    # for every head word compute weighted average of desc words
    desc_avg_word = K.batch_dot(activation_weights, desc_words, axes=(2,1))
    return K.concatenate((desc_avg_word, head_words))


class SimpleContext(Lambda):
    def __init__(self,**kwargs):
        super(SimpleContext, self).__init__(simple_context,**kwargs)
        self.supports_masking = True

    def compute_mask(self, input, input_mask=None):
        return input_mask[:, maxlend:]
    
    def get_output_shape_for(self, input_shape):
        nb_samples = input_shape[0]
        n = 2*(rnn_size - activation_rnn_size)
        return (nb_samples, maxlenh, n)

In [616]:
#if activation_rnn_size:
    #model.add(SimpleContext(name='simplecontext_1'))
model.add(TimeDistributed(Dense(vocab_size,
                                W_regularizer=regularizer, b_regularizer=regularizer,
                                name = 'timedistributed_1')))
model.add(Activation('softmax', name='activation_1'))

C:\Users\rasik\Anaconda3\envs\mlenv\lib\site-packages\ipykernel\__main__.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(29467, name="timedistributed_1", kernel_regularizer=None, bias_regularizer=None)`


In [617]:
from keras.optimizers import Adam, RMSprop # usually I prefer Adam but article used rmsprop
# opt = Adam(lr=LR)  # keep calm and reduce learning rate
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
K.set_value(model.optimizer.lr,np.float32(LR))

In [ ]:
def str_shape(x):
    return 'x'.join(map(str,x.shape))

def lpadd(x, maxlend=maxlend, eos=eos):
    """left (pre) pad a description to maxlend and then add eos.
    The eos is the input to predicting the first word in the headline
    """
    assert maxlend >= 0
    if maxlend == 0:
        return [eos]
    n = len(x)
    if n > maxlend:
        x = x[-maxlend:]
        n = maxlend
    return [empty]*(maxlend-n) + x + [eos]

def vocab_fold(xs):
    """convert list of word indexes that may contain words outside vocab_size to words inside.
    If a word is outside, try first to use glove_idx2idx to find a similar word inside.
    If none exist then replace all accurancies of the same unknown word with <0>, <1>, ...
    """
    xs = [x if x < oov0 else glove_idx2idx.get(x,x) for x in xs]
    # the more popular word is <0> and so on
    outside = sorted([x for x in xs if x >= oov0])
    # if there are more than nb_unknown_words oov words then put them all in nb_unknown_words-1
    outside = dict((x,vocab_size-1-min(i, nb_unknown_words-1)) for i, x in enumerate(outside))
    xs = [outside.get(x,x) for x in xs]
    return xs

In [ ]:
def flip_headline(x, nflips=None, model=None, debug=False):
    """given a vectorized input (after `pad_sequences`) flip some of the words in the second half (headline)
    with words predicted by the model
    """
    if nflips is None or model is None or nflips <= 0:
        return x
    
    batch_size = len(x)
    assert np.all(x[:,maxlend] == eos)
    probs = model.predict(x, verbose=0, batch_size=batch_size)
    x_out = x.copy()
    for b in range(batch_size):
        # pick locations we want to flip
        # 0...maxlend-1 are descriptions and should be fixed
        # maxlend is eos and should be fixed
        flips = sorted(random.sample(range(maxlend+1,maxlen), nflips))
        for input_idx in flips:
            if x[b,input_idx] == empty or x[b,input_idx] == eos:
                continue
            # convert from input location to label location
            # the output at maxlend (when input is eos) is feed as input at maxlend+1
            label_idx = input_idx - (maxlend+1)
            prob = probs[b, label_idx]
            w = prob.argmax()
            if w == empty:  # replace accidental empty with oov
                w = oov0
            x_out[b,input_idx] = w
    return x_out 

In [ ]:
def conv_seq_labels(xds, xhs, nflips=None, model=None, debug=False):
    """description and hedlines are converted to padded input vectors. headlines are one-hot to label"""
    batch_size = len(xhs)
    assert len(xds) == batch_size
    x = [vocab_fold(lpadd(xd)+xh) for xd,xh in zip(xds,xhs)]  # the input does not have 2nd eos
    x = sequence.pad_sequences(x, maxlen=maxlen, value=empty, padding='post', truncating='post')
    x = flip_headline(x, nflips=nflips, model=model, debug=debug)
    
    y = np.zeros((batch_size, maxlenh, vocab_size))
    for i, xh in enumerate(xhs):
        xh = vocab_fold(xh) + [eos] + [empty]*maxlenh  # output does have a eos at end
        xh = xh[:maxlenh]
        y[i,:,:] = np_utils.to_categorical(xh, vocab_size)
        
    return x, y

In [ ]:
def gen(Xd, Xh, batch_size=batch_size, nb_batches=None, nflips=None, model=None, debug=False, seed=0):
    """yield batches. for training use nb_batches=None
    for validation generate deterministic results repeating every nb_batches
    
    while training it is good idea to flip once in a while the values of the headlines from the
    value taken from Xh to value generated by the model.
    """
    c = nb_batches if nb_batches else 0
    while True:
        xds = []
        xhs = []
        if nb_batches and c >= nb_batches:
            c = 0
        new_seed = random.randint(0, sys.maxsize)
        random.seed(c+123456789)
        for b in range(batch_size):
            t = random.randint(0,len(Xd)-1)

            xd = Xd[t]
            s = random.randint(min(maxlend,len(xd)), max(maxlend,len(xd)))
            xds.append(xd[:s])
            
            xh = Xh[t]
            s = random.randint(min(maxlenh,len(xh)), max(maxlenh,len(xh)))
            xhs.append(xh[:s])

        # undo the seeding before we yield inorder not to affect the caller
        c+= 1
        random.seed(new_seed)

        yield conv_seq_labels(xds, xhs, nflips=nflips, model=model, debug=debug)
        
#r = next(gen(X_train, Y_train, batch_size=batch_size))
#r[0].shape, r[1].shape, len(r)

def prt(label, x):
    print(label+':')
    for w in x:
        print(idx2word[w])
    print('\n')

def test_gen(gen, n=5):
    Xtr,Ytr = next(gen)
    for i in range(n):
        assert Xtr[i,maxlend] == eos
        x = Xtr[i,:maxlend]
        y = Xtr[i,maxlend:]
        yy = Ytr[i,:]
        yy = np.where(yy)[1]
        prt('L',yy)
        prt('H',y)
        if maxlend:
            prt('D',x)
            
#test_gen(gen(X_train, Y_train, batch_size=batch_size))

In [ ]:
history = {}

In [ ]:
traingen = gen(X_train, Y_train, batch_size=batch_size, nflips=nflips, model=model)
valgen = gen(X_test, Y_test, nb_batches=nb_val_samples//batch_size, batch_size=batch_size)

r = next(traingen)
r[0].shape, r[1].shape, len(r)

((64, 25), (64, 25, 29467), 2)

In [ ]:
for iteration in range(100):
    h = model.fit_generator(traingen, samples_per_epoch=nb_train_samples,
                        nb_epoch=1, validation_data=valgen, nb_val_samples=nb_val_samples
                           )

C:\Users\rasik\Anaconda3\envs\mlenv\lib\site-packages\ipykernel\__main__.py:3: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  app.launch_new_instance()
C:\Users\rasik\Anaconda3\envs\mlenv\lib\site-packages\ipykernel\__main__.py:3: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=<generator..., steps_per_epoch=20, epochs=1, validation_steps=20)`
  app.launch_new_instance()


Epoch 1/1
20/20 [==============================] - 288s 14s/step - loss: 10.2863 - val_loss: 10.2682
Epoch 1/1
20/20 [==============================] - 265s 13s/step - loss: 9.9692 - val_loss: 9.3495
Epoch 1/1
20/20 [==============================] - 257s 13s/step - loss: 8.4662 - val_loss: 7.9845
Epoch 1/1
20/20 [==============================] - 257s 13s/step - loss: 7.6213 - val_loss: 7.6910
Epoch 1/1
20/20 [==============================] - 258s 13s/step - loss: 7.4157 - val_loss: 7.6391
Epoch 1/1
20/20 [==============================] - 254s 13s/step - loss: 7.3620 - val_loss: 7.5664
Epoch 1/1
19/20 [===========================>..] - ETA: 10s - loss: 7.3093